In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json

In [2]:
# load dataset
dataset_file_path = r"C:\Users\TARUN\Desktop\SDG_6_Indicators_Aligned.csv"
df = pd.read_csv(dataset_file_path, header=0, low_memory=False, encoding='utf-8').reset_index(drop=True)
# Get the directory where the file is located
directory_path = os.path.dirname(dataset_file_path)
print("Directory path:", directory_path)

# Check missing values
print("\nMissing values per column:\n", df.isnull().sum())

Directory path: C:\Users\TARUN\Desktop

Missing values per column:
 Country           0
Indicator         0
Indicator_Type    0
SeriesCode        0
Units             0
2000              0
2001              0
2002              0
2003              0
2004              0
2005              0
2006              0
2007              0
2008              0
2009              0
2010              0
2011              0
2012              0
2013              0
2014              0
2015              0
2016              0
2017              0
2018              0
2019              0
2020              0
2021              0
2022              0
2023              0
Role              0
dtype: int64


In [3]:
# Create a mapping for countries and indicators to unique numeric codes
country_code_map = {country: idx for idx, country in enumerate(df['Country'].unique())}
indicator_code_map = {indicator: idx for idx, indicator in enumerate(df['Indicator'].unique())}

# Add those codes to the dataframe
df['country_code'] = df['Country'].map(country_code_map)
df['indicator_code'] = df['Indicator'].map(indicator_code_map)

In [4]:
# Print original shape
print("Original shape:")
print(df.shape)

# Drop duplicate rows (across all columns including years, role, etc.)
df = df.drop_duplicates()

# Print new shape after dropping duplicates
print("New shape after dropping duplicates:")
print(df.shape)

Original shape:
(683, 32)
New shape after dropping duplicates:
(683, 32)


In [5]:
import pycountry
from geopy.geocoders import ArcGIS
from geopy.extra.rate_limiter import RateLimiter

# --------------------------------------------
# Step 2: Define Helper Functions
# --------------------------------------------
def remove_third_element(point):
    """Removes the third element from a tuple if it exists."""
    if isinstance(point, tuple):
        return point[:2]
    else:
        return point

def get_code(name):
    """Gets the 2-letter ISO country code for a given country name."""
    try:
        return pycountry.countries.lookup(name).alpha_2
    except LookupError:
        return None

# --------------------------------------------
# Step 3: Setup Geocoder
# --------------------------------------------
geolocator = ArcGIS(user_agent="SDG6-Encoder")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def get_location(row, country_col, code_col):
    """Returns a geopy location object from country name and code."""
    country_name = row[country_col]
    country_code = row[code_col]
    if country_code:
        return geocode(f"{country_name}, {country_code}")
    else:
        return geocode(country_name)

# --------------------------------------------
# Step 4: Geocode Unique Countries
# --------------------------------------------
unique_countries = pd.DataFrame(df['Country'].unique(), columns=['Country'])
unique_countries['country_code'] = unique_countries['Country'].apply(get_code)
unique_countries['location'] = unique_countries.apply(lambda row: get_location(row, 'Country', 'country_code'), axis=1)
unique_countries['point'] = unique_countries['location'].apply(lambda loc: (loc.latitude, loc.longitude) if loc else None)
unique_countries['point'] = unique_countries['point'].apply(remove_third_element)
unique_countries['latitude'] = unique_countries['point'].apply(lambda x: x[0] if x else None)
unique_countries['longitude'] = unique_countries['point'].apply(lambda x: x[1] if x else None)

# --------------------------------------------
# Step 5: Drop Existing Geo Columns if Present
# --------------------------------------------
df = df.drop(columns=['country_code', 'latitude', 'longitude'], errors='ignore')

# --------------------------------------------
# Step 6: Merge Geocoded Data Back to Main DataFrame
# --------------------------------------------
df = df.merge(unique_countries[['Country', 'location', 'country_code', 'latitude', 'longitude', 'point']], on='Country', how='left')

# --------------------------------------------
# Step 7: Preview
# --------------------------------------------
print(df[['Country', 'country_code', 'latitude', 'longitude']].drop_duplicates().head())
df.head()


    Country country_code   latitude  longitude
0   Austria           AT  46.633290  14.310900
1    France           FR  46.559417   2.550540
2   Germany           DE  51.110631  10.392278
3  Paraguay           PY -23.219812 -58.410666
4      Peru           PE -10.000000 -76.000000


,Country,Indicator,Indicator_Type,SeriesCode,Units,2000,2001,2002,2003,2004,...,2021,2022,2023,Role,indicator_code,location,country_code,latitude,longitude,point
0,Austria,6.3.1,Proportion of safely treated domestic wastewat...,EN_WWT_WWDS,PERCENT,0.0,0.0,0.0,0.0,0.0,...,0.0,98.20522,0.0,Donor,0,"(Austria, (46.63329, 14.3109))",AT,46.633290,14.310900,"(46.63329, 14.3109)"
1,France,6.3.1,Proportion of safely treated domestic wastewat...,EN_WWT_WWDS,PERCENT,0.0,0.0,0.0,0.0,0.0,...,0.0,87.94422,0.0,Donor,0,"(France, (46.559417044, 2.550539953))",FR,46.559417,2.550540,"(46.559417044, 2.550539953)"
2,Germany,6.3.1,Proportion of safely treated domestic wastewat...,EN_WWT_WWDS,PERCENT,0.0,0.0,0.0,0.0,0.0,...,0.0,98.96163,0.0,Donor,0,"(Germany, (51.110631049, 10.392277932))",DE,51.110631,10.392278,"(51.110631049, 10.392277932)"
3,Paraguay,6.3.1,Proportion of safely treated domestic wastewat...,EN_WWT_WWDS,PERCENT,0.0,0.0,0.0,0.0,0.0,...,0.0,25.14752,0.0,Donor,0,"(Paraguay, (-23.219812096, -58.410666019))",PY,-23.219812,-58.410666,"(-23.219812096, -58.410666019)"
4,Peru,6.3.1,Proportion of safely treated domestic wastewat...,EN_WWT_WWDS,PERCENT,0.0,0.0,0.0,0.0,0.0,...,0.0,48.65280,0.0,Donor,0,"(Perú, (-10.0, -76.0))",PE,-10.000000,-76.000000,"(-10.0, -76.0)"


In [6]:
import os

# Step 1: Choose directory to save in
directory_path = r"C:\Users\TARUN\Desktop" 

# Step 2: Define the output path
geo_path = os.path.join(directory_path, 'GeoLocationInfo.csv')

# Step 3: Save the new geo-encoded DataFrame
df.to_csv(geo_path, index=False, encoding='utf-8')

print(f"GeoLocationInfo.csv saved for the NEW dataset at:\n{geo_path}")


GeoLocationInfo.csv saved for the NEW dataset at:
C:\Users\TARUN\Desktop\GeoLocationInfo.csv


In [7]:
import pandas as pd

# Step 1: Load the previously saved full GeoLocationInfo file
geo_df = pd.read_csv(r"C:\Users\TARUN\Desktop\GeoLocationInfo.csv")  

# Step 2: Specify the columns to retain
columns_to_keep = [
    'Country', 'Indicator', 'Indicator_Type', 'SeriesCode', 'Units',
    '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
    '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
    '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023',
    'country_code', 'indicator_code', 'location', 'point',
    'latitude', 'longitude', 'Role'
]

# Step 3: Filter and rename columns
selected_geo_df = geo_df[columns_to_keep]
# print(selected_geo_df)  # Uncomment if you want to preview

# Step 4: Save the final cleaned file
selected_geo_df.to_csv(r"C:\Users\TARUN\Desktop\GeoLocationInfo_Final.csv", index=False, encoding='utf-8')

# Step 5: Display the result
print(selected_geo_df.head())


    Country Indicator                                     Indicator_Type  \
0   Austria     6.3.1  Proportion of safely treated domestic wastewat...   
1    France     6.3.1  Proportion of safely treated domestic wastewat...   
2   Germany     6.3.1  Proportion of safely treated domestic wastewat...   
3  Paraguay     6.3.1  Proportion of safely treated domestic wastewat...   
4      Peru     6.3.1  Proportion of safely treated domestic wastewat...   

    SeriesCode    Units  2000  2001  2002  2003  2004  ...  2021      2022  \
0  EN_WWT_WWDS  PERCENT   0.0   0.0   0.0   0.0   0.0  ...   0.0  98.20522   
1  EN_WWT_WWDS  PERCENT   0.0   0.0   0.0   0.0   0.0  ...   0.0  87.94422   
2  EN_WWT_WWDS  PERCENT   0.0   0.0   0.0   0.0   0.0  ...   0.0  98.96163   
3  EN_WWT_WWDS  PERCENT   0.0   0.0   0.0   0.0   0.0  ...   0.0  25.14752   
4  EN_WWT_WWDS  PERCENT   0.0   0.0   0.0   0.0   0.0  ...   0.0  48.65280   

   2023  country_code  indicator_code  location  \
0   0.0            AT  

In [8]:
import pandas as pd
import pycountry

# Step 1: Load the dataset
df = pd.read_csv(r"C:\Users\TARUN\Desktop\GeoLocationInfo_Final.csv")  # <-- your correct path

# Step 2: Create a correction mapping for problematic country names
country_name_fixes = {
    "Bolivia (Plurinational State of)": "Bolivia",
    "China, Hong Kong Special Administrative Region": "China",
    "China, Macao Special Administrative Region": "China",
    "Iran (Islamic Republic of)": "Iran",
    "Micronesia (Federated States of)": "Federated States of Micronesia",
    "Republic of Korea": "South Korea",
    "Republic of Moldova": "Moldova",
    "Russian Federation": "Russia",
    "Syrian Arab Republic": "Syria",
    "United Republic of Tanzania": "Tanzania",
    "Venezuela (Bolivarian Republic of)": "Venezuela",
    "Viet Nam": "Vietnam",
    "Kosovo": "XK",  # Kosovo isn't in pycountry; assign known ISO-like code
    "Holy See": "VA",  # Vatican
    "Iraq (Central Iraq)": "Iraq",
    "Iraq (Kurdistan Region)": "Iraq",
    "Netherlands (Kingdom of the)": "Netherlands",
    "State of Palestine": "PS",
    "Saint Helena": "SH",  # Not always in pycountry; ISO code is SH
    "United Kingdom (England and Wales)": "United Kingdom",
    "United Kingdom (Northern Ireland)": "United Kingdom",
    "United Kingdom (Scotland)": "United Kingdom",
    "United States Virgin Islands": "Virgin Islands, U.S.",
    "Democratic Republic of the Congo": "Congo, The Democratic Republic of the",
    "Channel Islands": "Jersey",  # or use direct code: 'JE'
    "Wallis and Futuna Islands": "Wallis and Futuna",
    "Other non-specified areas in Eastern Asia": "OTH"  # No official ISO code
    # Add more mappings as needed
}

# Step 3: Define a safe lookup function with fallback
def safe_country_code(name):
    if name in country_name_fixes:
        fixed = country_name_fixes[name]
        if len(fixed) == 2:  # Already an ISO code (e.g., XK, VA)
            return fixed
        try:
            return pycountry.countries.lookup(fixed).alpha_2
        except LookupError:
            return None
    else:
        try:
            return pycountry.countries.lookup(name).alpha_2
        except LookupError:
            return None

# Step 4: Fill in missing country_code values
df['country_code'] = df.apply(
    lambda row: row['country_code'] if pd.notna(row['country_code']) else safe_country_code(row['Country']),
    axis=1
)

# Step 5: Save updated DataFrame
df.to_csv(r"C:\Users\TARUN\Desktop\GeoLocationInfo_Final_Fixed.csv", index=False, encoding='utf-8')

# Optional: Preview fixed rows
print(df[df['country_code'].isna()][['Country', 'country_code']])


                             Country country_code
17                              Asia         None
19         Australia and New Zealand         None
34                         Caribbean         None
35                   Central America         None
36                      Central Asia         None
..                               ...          ...
599  Southern Asia (excluding India)         None
600                  Southern Europe         None
602               Sub-Saharan Africa         None
619                   Western Africa         None
620                     Western Asia         None

[105 rows x 2 columns]


In [9]:
import pandas as pd
import pycountry

# Step 1: Load the updated file
df = pd.read_csv(r"C:\Users\TARUN\Desktop\GeoLocationInfo_Final_Fixed.csv")  

# Step 2: Manual overrides for known missing ISO entries
country_name_fixes = {
    "Democratic Republic of the Congo": "Congo, The Democratic Republic of the",
    "United States Virgin Islands": "Virgin Islands, U.S.",
    "Wallis and Futuna Islands": "Wallis and Futuna",
    "Channel Islands": "Jersey",
    "Netherlands (Kingdom of the)": "Netherlands",
    "Kosovo": "XK",
    "Holy See": "VA",
    "Micronesia (Federated States of)": "Federated States of Micronesia",
    "State of Palestine": "Palestine",
    "United Kingdom (England and Wales)": "United Kingdom",
    "United Kingdom (Northern Ireland)": "United Kingdom",
    "United Kingdom (Scotland)": "United Kingdom"
}

manual_code_override = {
    "State of Palestine": "PS",
    "Channel Islands": "JE",
    "Wallis and Futuna Islands": "WF",
    "United States Virgin Islands": "VI",
    "Kosovo": "XK",
    "Holy See": "VA"
}

# Step 3: Safe lookup function with fallbacks
def safe_country_code(name):
    if name in manual_code_override:
        return manual_code_override[name]
    elif name in country_name_fixes:
        fixed = country_name_fixes[name]
        if len(fixed) == 2:
            return fixed
        try:
            return pycountry.countries.lookup(fixed).alpha_2
        except LookupError:
            return None
    else:
        try:
            return pycountry.countries.lookup(name).alpha_2
        except LookupError:
            return None

# Step 4: Apply corrections only where country_code is missing
df['country_code'] = df.apply(
    lambda row: row['country_code'] if pd.notna(row['country_code']) else safe_country_code(row['Country']),
    axis=1
)

# Step 5: Tag any remaining null country codes as 'REGION'
df['country_code'] = df['country_code'].fillna('REGION')

# Step 6: Save the final corrected version
df.to_csv(r"C:\Users\TARUN\Desktop\GeoLocationInfo_Final_Tagged.csv", index=False, encoding='utf-8')

# Step 7: Preview
print(df[['Country', 'country_code']].head(10))


       Country country_code
0      Austria           AT
1       France           FR
2      Germany           DE
3     Paraguay           PY
4         Peru           PE
5  Philippines           PH
6     Portugal           PT
7        Samoa           WS
8        Spain           ES
9       Sweden           SE


In [10]:
import pandas as pd
import os

# STEP 1: Load the already geoencoded dataset
geo_df = pd.read_csv(r"C:\Users\TARUN\Desktop\GeoLocationInfo_Final_Tagged.csv")

# STEP 2: Ensure consistent column naming
# If not already present, create a simplified "country_code" column for joining
if 'country_code' not in geo_df.columns:
    geo_df['country_code'] = pd.factorize(geo_df['Country'])[0] + 1

# STEP 3: Separate donors and recipients
donors_df = geo_df[geo_df['Role'] == 'Donor'].drop_duplicates(subset='Country')[['Country', 'latitude', 'longitude']].copy()
recipients_df = geo_df[geo_df['Role'] == 'Recipient'].drop_duplicates(subset='Country')[['Country', 'latitude', 'longitude']].copy()

# STEP 4: Rename columns to make them distinct
donors_df = donors_df.rename(columns={
    'latitude': 'donor_lat',
    'longitude': 'donor_long'
})

recipients_df = recipients_df.rename(columns={
    'latitude': 'recipient_lat',
    'longitude': 'recipient_long'
})

# STEP 5: Merge coordinates back into the main dataframe
merged_df = geo_df.copy()
merged_df = merged_df.merge(donors_df, on='Country', how='left')
merged_df = merged_df.merge(recipients_df, on='Country', how='left')

# STEP 6: Drop duplicate rows if any were introduced
merged_df = merged_df.drop_duplicates()

# STEP 7: Assign null values with 0
merged_df.fillna(0, inplace=True)

# STEP 8: Save the result
save_path = os.path.join(r"C:\Users\TARUN\Desktop\GeoLocationInfo_Merged_With_Roles.csv")
merged_df.to_csv(save_path, index=False)


In [11]:
import pandas as pd
import os

# STEP 1: Load the SDG-6 geolocation-enhanced dataset
geo_path = os.path.join(r"C:\Users\TARUN\Desktop", 'GeoLocationInfo_Merged_With_Roles.csv') 
temp_df = pd.read_csv(geo_path, encoding="utf-8")

# STEP 2: Sort for consistency
temp_df = temp_df.sort_values(by=['Country', 'Role'], ascending=[True, True])

# STEP 3: Extract donor and recipient country lists
donor_countries_list = temp_df[temp_df['Role'] == 'Donor']['Country'].unique()
recipient_countries_list = temp_df[temp_df['Role'] == 'Recipient']['Country'].unique()

# STEP 4: Print useful context
print("Number of donor countries:", len(donor_countries_list))
print("Number of recipient countries:", len(recipient_countries_list))


Number of donor countries: 26
Number of recipient countries: 73


In [12]:
import pandas as pd
import os

# STEP 1: Load the geoencoded SDG-6 dataset
geo_path = os.path.join(r"C:\Users\TARUN\Desktop", 'GeoLocationInfo_Merged_With_Roles.csv')
df = pd.read_csv(geo_path, encoding="utf-8")

# STEP 2: Melt the year columns (2000–2023) to long format
year_columns = [str(year) for year in range(2000, 2024)]
df_melted = df.melt(
    id_vars=[
        "Country", "Indicator", "Indicator_Type", "SeriesCode", "Units",
        "Role", "country_code", "latitude", "longitude",
        "donor_lat", "donor_long", "recipient_lat", "recipient_long"  # Added the new lat/long columns
    ],
    value_vars=year_columns,
    var_name="year",
    value_name="Value"
)

# STEP 3: Fill NaN with 0
df_melted.fillna(0, inplace=True)

# STEP 4: Convert year to int and create 3-year bins
df_melted["year"] = df_melted["year"].astype(int)
min_year = df_melted["year"].min()
max_year = df_melted["year"].max()

# Generate 3-year bin intervals
bin_ranges = []
current = min_year
while current <= max_year:
    end = current + 2   # <-- 3-year binning: +2
    bin_ranges.append((current, end))
    current += 3        # <-- Step +3 years

def map_to_3yr_bin(year):
    for start, end in bin_ranges:
        if start <= year <= end:
            return f"{start}-{end}"
    return "Other"

df_melted["year_interval"] = df_melted["year"].apply(map_to_3yr_bin)

# STEP 5: Normalize 'Value' column per Indicator_Type
df_melted["normalized_value"] = 0.0
for indicator in df_melted["Indicator_Type"].unique():
    mask = df_melted["Indicator_Type"] == indicator
    min_val = df_melted.loc[mask, "Value"].min()
    max_val = df_melted.loc[mask, "Value"].max()
    df_melted.loc[mask, "normalized_value"] = df_melted.loc[mask, "Value"].apply(
        lambda x: 0.01 + ((x - min_val) / (max_val - min_val) * (2 - 0.01)) if max_val != min_val else 1.0
    )

# STEP 6: Assign ring_location based on year_interval
def assign_ring_locations(df_input):
    intervals = sorted(df_input["year_interval"].unique())
    ring_location_dict = {interval: -135 + i * 35 for i, interval in enumerate(intervals)}
    df_input["ring_location"] = df_input["year_interval"].map(ring_location_dict)
    return df_input

df_melted = assign_ring_locations(df_melted)

# STEP 7: Save the final normalized file
norm_path = os.path.join(r"C:\Users\TARUN\Desktop", 'Normalized_SDG6_Data_Geo_Final.csv')
df_melted.to_csv(norm_path, index=False, encoding='utf-8')


In [13]:
import pandas as pd
import time
import os

# Load normalized dataset
normalized_df = pd.read_csv(r"C:\Users\TARUN\Desktop\Normalized_SDG6_Data_Geo_Final.csv")

# Load node and tag templates
node_template = pd.read_csv(r"C:\Users\TARUN\Desktop\np_node-template.csv")  
tag_template = pd.read_csv(r"C:\Users\TARUN\Desktop\Tag_Template_SDG-6.csv")
Colors = pd.read_csv(r"C:\Users\TARUN\Desktop\colors.csv")

In [14]:
# Initialize node and tag DataFrames
node_df = pd.DataFrame(columns=node_template.columns)
tag_df = pd.DataFrame(columns=tag_template.columns)

In [15]:
# Assign node IDs to countries
country_node_map = {c: i + 1 for i, c in enumerate(normalized_df["Country"].unique())}
normalized_df["node_id"] = normalized_df["Country"].map(country_node_map)

# Define z_layer assignment based on Role
def z_layer(role):
    return 20 if role == "Donor" else 0 if role == "Recipient" else 10  
normalized_df["z_layer"] = normalized_df["Role"].apply(z_layer)


In [16]:
normalized_df

,Country,Indicator,Indicator_Type,SeriesCode,Units,Role,country_code,latitude,longitude,donor_lat,donor_long,recipient_lat,recipient_long,year,Value,year_interval,normalized_value,ring_location,node_id,z_layer
0,Austria,6.3.1,Proportion of safely treated domestic wastewat...,EN_WWT_WWDS,PERCENT,Donor,AT,46.633290,14.310900,46.633290,14.310900,0.000000,0.000000,2000,0.0,2000-2005,0.01,-135,1,20
1,France,6.3.1,Proportion of safely treated domestic wastewat...,EN_WWT_WWDS,PERCENT,Donor,FR,46.559417,2.550540,46.559417,2.550540,0.000000,0.000000,2000,0.0,2000-2005,0.01,-135,2,20
2,Germany,6.3.1,Proportion of safely treated domestic wastewat...,EN_WWT_WWDS,PERCENT,Donor,DE,51.110631,10.392278,51.110631,10.392278,0.000000,0.000000,2000,0.0,2000-2005,0.01,-135,3,20
3,Paraguay,6.3.1,Proportion of safely treated domestic wastewat...,EN_WWT_WWDS,PERCENT,Donor,PY,-23.219812,-58.410666,-23.219812,-58.410666,0.000000,0.000000,2000,0.0,2000-2005,0.01,-135,4,20
4,Peru,6.3.1,Proportion of safely treated domestic wastewat...,EN_WWT_WWDS,PERCENT,Donor,PE,-10.000000,-76.000000,-10.000000,-76.000000,0.000000,0.000000,2000,0.0,2000-2005,0.01,-135,5,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16387,Uzbekistan,6.a.1,Total official development assistance (gross d...,DC_TOF_WASHL,CON_USD_M,Recipient,UZ,41.581346,63.421883,0.000000,0.000000,41.581346,63.421883,2023,0.0,2018-2023,0.01,-30,168,0
16388,Vanuatu,6.a.1,Total official development assistance (gross d...,DC_TOF_WASHL,CON_USD_M,Recipient,VU,-15.241356,166.872757,0.000000,0.000000,-15.241356,166.872757,2023,0.0,2018-2023,0.01,-30,234,0
16389,Yemen,6.a.1,Total official development assistance (gross d...,DC_TOF_WASHL,CON_USD_M,Recipient,YE,15.905206,47.593953,0.000000,0.000000,15.905206,47.593953,2023,0.0,2018-2023,0.01,-30,183,0
16390,Zambia,6.a.1,Total official development assistance (gross d...,DC_TOF_WASHL,CON_USD_M,Recipient,ZM,-14.468804,28.767973,0.000000,0.000000,-14.468804,28.767973,2023,0.0,2018-2023,0.01,-30,204,0


In [17]:
# Define SDG-6 Specific Color Map
sdg_color_map = {
    '6.3.1': [0, 255, 255], 
    '6.3.2': [152, 0, 255], 
    '6.5.1': [0, 255, 255], 
    '6.a.1': [255, 0, 255]
}

# Initialize ring_id and tag id counter
ring_id = normalized_df["node_id"].max() + 1  
np_tag_id_counter = ring_id + 100000  


In [18]:
# === FUNCTIONS ===

def create_country_node_tag(node_id, lat, long, z, colors, title, description):
    global node_df, tag_df, np_tag_id_counter
    pin = node_template.iloc[0].copy()
    tag = tag_template.iloc[0].copy()
    np_tag_id_counter += 1
    pin["np_node_id"] = node_id
    pin["np_tag_id"] = np_tag_id_counter
    pin["record_id"] = node_id
    pin.update({
        'np_table_id': 1, 'parent_id': 0, 'scale_x': 0.5, 'scale_y': 0.5, 'scale_z': 0.5,
        'translate_x': long, 'translate_y': lat, 'translate_z': z, 'np_geometry_id': 19,
        'np_topo_id': 6, 'np_color_id': 1, 'color_r': colors[0], 'color_g': colors[1], 'color_b': colors[2]
    })
    tag["np_tag_id"] = np_tag_id_counter
    tag["record_id"] = node_id
    tag.update({'table_id': 1, 'title': title, 'description': description})
    node_df = pd.concat([node_df, pd.DataFrame([pin])], ignore_index=True)
    tag_df = pd.concat([tag_df, pd.DataFrame([tag])], ignore_index=True)

def create_year_node_tag(ring_location, parent_id, ring_id, title):
    global node_df, tag_df, np_tag_id_counter
    pin = node_template.iloc[1].copy()
    tag = tag_template.iloc[0].copy()
    np_tag_id_counter += 1
    pin["np_node_id"] = ring_id
    pin["np_tag_id"] = np_tag_id_counter
    pin["record_id"] = ring_id
    pin.update({
        'parent_id': parent_id, 'branch_level': 2, 'translate_x': ring_location,
        'scale_x': 0.5, 'scale_y': 0.5, 'scale_z': 0.5, 'np_table_id': 1, 'np_color_id': 20,
        'color_r': 55, 'color_g': 190, 'color_b': 190
    })
    tag["np_tag_id"] = np_tag_id_counter
    tag["record_id"] = ring_id
    tag.update({'table_id': 1, 'title': title, 'description': 'Year Ring'})
    node_df = pd.concat([node_df, pd.DataFrame([pin])], ignore_index=True)
    tag_df = pd.concat([tag_df, pd.DataFrame([tag])], ignore_index=True)

def create_petal_rings(petal_id, parent_id, location, title, colors):
    global node_df, tag_df, np_tag_id_counter
    pin = node_template.iloc[1].copy()
    tag = tag_template.iloc[0].copy()
    np_tag_id_counter += 1
    pin["np_node_id"] = petal_id
    pin["np_tag_id"] = np_tag_id_counter
    pin["record_id"] = petal_id
    pin["np_data_id"] = petal_id
    pin.update({
        'parent_id': parent_id, 'branch_level': 3, 'translate_x': location,
        'scale_x': 1, 'scale_y': 1, 'scale_z': 1, 'np_table_id': 1, 'np_topo_id': 3,
        'ratio': 0.1, 'color_r': colors[0], 'color_g': colors[1], 'color_b': colors[2]
    })
    tag["np_tag_id"] = np_tag_id_counter
    tag["record_id"] = petal_id
    tag.update({'table_id': 1, 'title': title, 'description': 'SDG'})
    node_df = pd.concat([node_df, pd.DataFrame([pin])], ignore_index=True)
    tag_df = pd.concat([tag_df, pd.DataFrame([tag])], ignore_index=True)

def link_nodes(link_id, parent_id, child_id, title, colors, ratio):
    global node_df, tag_df, np_tag_id_counter
    pin = node_template.iloc[0].copy()
    tag = tag_template.iloc[0].copy()
    np_tag_id_counter += 1
    pin["np_node_id"] = link_id
    pin["np_tag_id"] = np_tag_id_counter
    pin["record_id"] = link_id
    pin.update({
        'np_table_id': 1, 'np_geometry_id': 3, 'np_topo_id': 6, 'np_color_id': 20,
        'ratio': ratio, 'parent_id': parent_id, 'child_id': child_id, 'type': 7,
        'color_r': colors[0], 'color_g': colors[1], 'color_b': colors[2]
    })
    tag["np_tag_id"] = np_tag_id_counter
    tag["record_id"] = link_id
    tag.update({'table_id': 1, 'title': title, 'description': title})
    node_df = pd.concat([node_df, pd.DataFrame([pin])], ignore_index=True)
    tag_df = pd.concat([tag_df, pd.DataFrame([tag])], ignore_index=True)


In [21]:
# === GENERATION ===

donor_year_dict = {}
recipient_year_dict = {}
donor_countries_set = set()
recipient_countries_set = set()

for i, row in normalized_df.iterrows():
    country = row["Country"]
    role = row["Role"]
    interval = row["year_interval"]
    node_id = row["node_id"]
    lat = row["latitude"]
    lon = row["longitude"]
    z = row["z_layer"]
    indicator = row["Indicator"]
    ring_location = row["ring_location"]

    if role == "Donor":
        if country not in donor_countries_set:
            create_country_node_tag(node_id, lat, lon, z, [0, 225, 0], country, "Donor Country")
            donor_countries_set.add(country)
        if country not in donor_year_dict:
            donor_year_dict[country] = {}
        if interval not in donor_year_dict[country]:
            create_year_node_tag(ring_location, node_id, ring_id, f"{interval} (Donor)")
            donor_year_dict[country][interval] = []  
            petal_id = ring_id
            location = -45
            group_df = normalized_df[(normalized_df["Country"] == country) & (normalized_df["year_interval"] == interval)]
            for indicator_value in group_df["Indicator"].unique():
                petal_id += 1
                location += 30
                color = sdg_color_map.get(indicator_value, [0, 0, 0])
                create_petal_rings(petal_id, ring_id, location, indicator_value, color)
                donor_year_dict[country][interval].append({indicator_value: petal_id})
            ring_id = petal_id + 1

    elif role == "Recipient":
        if country not in recipient_countries_set:
            create_country_node_tag(node_id, lat, lon, z, [225, 0, 0], country, "Recipient Country")
            recipient_countries_set.add(country)
        if country not in recipient_year_dict:
            recipient_year_dict[country] = {}
        if interval not in recipient_year_dict[country]:
            create_year_node_tag(ring_location, node_id, ring_id, f"{interval} (Recipient)")
            recipient_year_dict[country][interval] = []  
            petal_id = ring_id
            location = -45
            group_df = normalized_df[(normalized_df["Country"] == country) & (normalized_df["year_interval"] == interval)]
            for indicator_value in group_df["Indicator"].unique():
                petal_id += 1
                location += 30
                color = sdg_color_map.get(indicator_value, [0, 0, 0])
                create_petal_rings(petal_id, ring_id, location, indicator_value, color)
                recipient_year_dict[country][interval].append({indicator_value: petal_id})
            ring_id = petal_id + 1


In [ ]:
link_id = ring_id + 1 

# Build fresh donor and recipient dictionaries from current node_df
donor_year_dict = {}
recipient_year_dict = {}

for _, row in normalized_df.iterrows():
    country = row["Country"]
    role = row["Role"]
    interval = row["year_interval"]
    indicator = row["Indicator"]
    node_id = row["node_id"]
    if node_id not in node_df["np_node_id"].values:
        continue
    target_dict = donor_year_dict if role == "Donor" else recipient_year_dict  
    if country not in target_dict:
        target_dict[country] = {}
    if interval not in target_dict[country]:
        target_dict[country][interval] = []
    target_dict[country][interval].append({indicator: node_id})

valid_ids = set(node_df["np_node_id"])

# Create links without any limit
for i, row in normalized_df.iterrows():
    interval = row["year_interval"]
    indicator = row["Indicator"]
    value = row["normalized_value"]
    for d_country, d_data in donor_year_dict.items():
        if interval not in d_data:
            continue
        for d_entry in d_data[interval]:
            d_id = d_entry.get(indicator)
            if not d_id or d_id not in valid_ids:
                continue
            for r_country, r_data in recipient_year_dict.items():
                if interval not in r_data:
                    continue
                for r_entry in r_data[interval]:
                    r_id = r_entry.get(indicator)
                    if not r_id or r_id not in valid_ids:
                        continue
                    link_nodes(
                        link_id,
                        parent_id=d_id,
                        child_id=r_id,
                        title=f"{indicator} | {d_country} → {r_country}",
                        colors=sdg_color_map.get(indicator, [0, 0, 0]),
                        ratio=value
                    )
                    link_id += 1


In [ ]:
# === FINAL CLEANUP ===

# Remove any rows with missing node IDs or tag IDs
node_df = node_df[node_df["np_node_id"].notnull()].copy()
tag_df = tag_df[tag_df["np_tag_id"].notnull()].copy()

# Define columns that should be integers
node_ints = [
    'np_node_id', 'type', 'np_data_id', 'selected', 'parent_id', 'branch_level', 'child_id', 'np_tag_id',
    'np_palette_id', 'np_ch_in_id', 'np_ch_out_id', 'ch_sync_time', 'np_palette_id_alt', 'np_color_id_alt',
    'np_material_id', 'np_geometry_id', 'np_color_id', 'color_fade', 'np_texture_id', 'hide', 'freeze',
    'np_topo_id', 'subspace', 'trigger_hi_x', 'trigger_hi_y', 'trigger_hi_z', 'trigger_lo_x', 'trigger_lo_y',
    'trigger_lo_z', 'proximity_x', 'proximity_y', 'proximity_z', 'proximity_mode_x', 'proximity_mode_y',
    'proximity_mode_z', 'segments_x', 'segments_y', 'segments_z', 'tag_mode', 'np_format_id', 'np_table_id', 'size'
]

# Convert specified columns to int type
node_df[node_ints] = node_df[node_ints].astype(int)

# Convert important ID columns to int64
node_df['record_id'] = node_df['record_id'].astype('int64')
tag_df[["np_tag_id", "record_id"]] = tag_df[["np_tag_id", "record_id"]].astype(int)


In [ ]:
# Save node and tag DataFrames to CSV
node_df.to_csv(r"C:\Users\TARUN\Desktop\np_node_SDG-6.csv", mode='w', index=False, encoding='utf-8', lineterminator='\r\n')
tag_df.to_csv(r"C:\Users\TARUN\Desktop\np_tag_SDG-6.csv", mode='w', index=False, encoding='utf-8', lineterminator='\r\n')
